In [27]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import os
import torch # type: ignore
import torch.nn as nn # type: ignore
from torch.utils.data import Dataset, DataLoader # type: ignore
import torch.optim as optim # type: ignore
from os.path import join
import datetime
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

%matplotlib inline

PATH_TO_DATA = os.path.join("../data", "Task3")

In [28]:
batch_size = 50
num_epochs = 500
learning_rate = 0.0001

In [29]:
full_df = pd.read_csv(os.path.join(PATH_TO_DATA, "housing.csv"))

In [30]:
full_df=full_df.dropna()
X=full_df.drop(columns=['median_house_value', 'ocean_proximity']).to_numpy(dtype=np.float32)
y=full_df['median_house_value'].to_numpy(dtype=np.float32)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [32]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).view(-1, 1).float()

X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).view(-1, 1).float()

datasets = torch.utils.data.TensorDataset(X_train, y_train)
train_iter = torch.utils.data.DataLoader(datasets, batch_size=64, shuffle=True)

In [33]:
class NeuralNet(nn.Module):

    def __init__(
        self, input_dimension, output_dimension, n_hidden_layers, neurons, retrain_seed, output_activation=None
    ):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        # self.activation = nn.Tanh()
        # self.activation = nn.LeakyReLU()
        self.activation=nn.ReLU()
        # self.output_activation=nn.ReLU()
        if output_activation is None:
            self.output_activation = nn.Identity()

        self.input_layer = nn.Linear(self.input_dimension, self.neurons)
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers - 1)]
        )
        self.output_layer = nn.Linear(self.neurons, self.output_dimension)
        self.retrain_seed = retrain_seed
        # Random Seed for weight initialization
        self.init_xavier()

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network
        # (see equation above)
        x = self.activation(self.input_layer(x))
        for k, l in enumerate(self.hidden_layers):
            x = self.activation(l(x))
        return self.output_activation(self.output_layer(x))

    def init_xavier(self):
        torch.manual_seed(self.retrain_seed)

        def init_weights(m):
            if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
                g = nn.init.calculate_gain("tanh")
                torch.nn.init.xavier_uniform_(m.weight, gain=g)
                # torch.nn.init.xavier_normal_(m.weight, gain=g)
                m.bias.data.fill_(0)

        self.apply(init_weights)

In [34]:
class CalHousing:
    def __init__(self, n_hidden_layers, n_neurons, train_df, target_df,X_valid,y_valid, seed):
        self.n_hidden_layers=n_hidden_layers
        self.n_neurons = n_neurons
        self.seed=seed
        self.device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_tensor = torch.tensor(train_df.values.astype(np.float32),dtype=torch.float32)
        target_tensor = torch.tensor(self.scale_targets(target_df), dtype=torch.float32)
        self.data = DataLoader(
            torch.utils.data.TensorDataset(train_tensor, target_tensor),
            batch_size=64,
            shuffle=False
        )
        self.model=NeuralNet(
            input_dimension=train_df.shape[1],
            output_dimension=1,
            n_hidden_layers=n_hidden_layers,
            neurons=n_neurons,
            retrain_seed=self.seed

        ).to(self.device)
        self.X=train_df
        self.y=target_df

        self.X_valid=X_valid
        self.y_valid=y_valid

        self.metadata_file=join("../logs","task_3" ,"metadata.json")

    def compute_loss(self, inputs, targets, verbose=True):
        preds=self.model(inputs)
        # targets=targets/10000
        res=targets-preds
        loss=torch.mean(res**2)
        # if verbose: print("Total loss: ", round(loss.item(), 4))
        return loss

    def save(self, loss_history):
        filename=join("..","models", "task_3", datetime.datetime.now().strftime("%m-%d %H:%M:%S")+".pt")
        salient_info={}
        salient_info["n_hidden_layers"]=self.n_hidden_layers
        salient_info["n_neurons"]=self.n_neurons
        salient_info["final_loss"]=loss_history[-1]
        salient_info["min_loss"]=min(loss_history)
        salient_info["model_path"]=filename
        salient_info["seed"]=self.seed

        torch.save(self.model.state_dict(),filename )
        with open(self.metadata_file, "a") as f:
            json.dump(salient_info, f)

    def scale_targets(self, target_df):
        y=target_df.values.astype(np.float32)
        self.scaler=StandardScaler()
        return(self.scaler.fit_transform(y.reshape(-1, 1)))
    
    # def validate(self, X,y):
    #     self.model.eval()
    #     with torch.no_grad():
    #         inputs=torch.tensor(X.values.astype(np.float32),dtype=torch.float32).to(self.device)
    #         targets=torch.tensor(self.scale_targets(y),dtype=torch.float32).to(self.device)
    #         loss=self.compute_loss(inputs, targets)
    #         return loss
    
    def RMSE(self, validation=True):
        if validation:
            X=self.X_valid
            y=self.y_valid
        else:
            X=self.X
            y=self.y
        self.model.eval()
        with torch.no_grad():
            inputs=torch.tensor(X.values.astype(np.float32),dtype=torch.float32).to(self.device)
            # targets=torch.tensor(self.scale_targets(y),dtype=torch.float32).to(self.device)
            preds=self.model(inputs)
            preds=self.scaler.inverse_transform(preds.cpu().numpy())
            loss=np.sqrt(np.mean((y.values-preds)**2))
            return loss



        

In [35]:
model=NeuralNet(8,1,4,40,13)
model.train()

NeuralNet(
  (activation): ReLU()
  (output_activation): Identity()
  (input_layer): Linear(in_features=8, out_features=40, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=40, out_features=40, bias=True)
  )
  (output_layer): Linear(in_features=40, out_features=1, bias=True)
)

In [36]:
criterion = nn.MSELoss(reduction='sum')

In [37]:
def train(model_inp, num_epochs = num_epochs):
    optimizer = torch.optim.RMSprop(model_inp.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for inputs, labels in train_iter:
            # forward pass
            outputs = model_inp(inputs)
            # defining loss
            loss = criterion(outputs, labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # computing gradients
            loss.backward()
            # accumulating running loss
            running_loss += loss.item()
            # updated weights based on computed gradients
            optimizer.step()
        if epoch % 20 == 0:    
            print('Epoch [%d]/[%d] running accumulative loss across all batches: %.3f' %
                  (epoch + 1, num_epochs, running_loss))
        running_loss = 0.0

In [38]:
train(model)

Epoch [1]/[500] running accumulative loss across all batches: 736113207148544.000
Epoch [21]/[500] running accumulative loss across all batches: 347004789063680.000
Epoch [41]/[500] running accumulative loss across all batches: 155752993652736.000
Epoch [61]/[500] running accumulative loss across all batches: 139697085710336.000
Epoch [81]/[500] running accumulative loss across all batches: 127406961786880.000
Epoch [101]/[500] running accumulative loss across all batches: 119016479227904.000
Epoch [121]/[500] running accumulative loss across all batches: 113181500342272.000
Epoch [141]/[500] running accumulative loss across all batches: 108844428427264.000
Epoch [161]/[500] running accumulative loss across all batches: 105158417825792.000
Epoch [181]/[500] running accumulative loss across all batches: 100869407145984.000
Epoch [201]/[500] running accumulative loss across all batches: 96608305922048.000
Epoch [221]/[500] running accumulative loss across all batches: 92689068687360.000


In [40]:
model.eval()
outputs = model(X_test)
err = np.sqrt(mean_squared_error(outputs.detach().numpy(), y_test.detach().numpy()))
print("Validation: ", err)
print("Training: ", np.sqrt(mean_squared_error(model(X_train).detach().numpy(), y_train.detach().numpy())))

Validation:  69748.086
Training:  69503.055
